In [1]:
import logging
import os
from typing import Optional
from boto3.session import Session

In [2]:
def aws_session(
    aws_key_env_var: Optional[str] = None, aws_secret_env_var: Optional[str] = None
) -> Session:
    """
    Creates an AWS session. Assumes that AWS credentials are stored in environment variables described at the top of
    this file. These environment variables are set up as part of our normal AWS server creation process.
    """
    if aws_key_env_var is None or aws_secret_env_var is None:
        return Session()
    aws_access_key_id = os.environ.get(aws_key_env_var, default=None)
    aws_secret_access_key = os.environ.get(aws_secret_env_var, default=None)
    if not aws_access_key_id or not aws_secret_access_key:
        raise EnvironmentError(
            f"One or both of {aws_key_env_var} and {aws_secret_env_var} are not set."
        )
    return Session(
        aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key
    )

def download_s3_to_directory(
    session: Session,
    bucket_name: str,
    s3_prefix: str,
    file_name: str,
    local_directory: str,
) -> None:
    """
    Download a file in the given local directory from S3 at a location equal to the s3_prefix + the file name.
    Sub-directories within the provided directory are ignored.
    :param session: handle to an authenticated AWS session
    :param bucket_name: S3 bucket name
    :param s3_prefix: S3 prefix of to-be-downloaded file
    :param local_directory: download file within this directory
    :param file_name
    :return: None
    """
    s3 = session.resource("s3")
    bucket = s3.Bucket(bucket_name)
    logging.info(
        f"Downloading S3 bucket {bucket_name} to local directory {local_directory} with prefix {s3_prefix}"
    )
    s3_location = f"{s3_prefix}/{file_name}"
    local_file_path = f"{local_directory}/{file_name}"
    bucket.download_file(s3_location, local_file_path)
    logging.info(f"Finished downloading file.")

In [3]:
naip_files=os.listdir("./data/naip_CA_2020_gridded/")

In [4]:
bucket_name = "vibrant-dragon"
s3_prefix   = "ca_rasters/naip_CA_2012_gridded"

for naip_file in naip_files:
    file_name = naip_file
    download_s3_to_directory(session=aws_session(), 
                             bucket_name=bucket_name, 
                             s3_prefix=s3_prefix, 
                             file_name=file_name, 
                             local_directory="data/naip_CA_2012_gridded/")